In [1]:
import requests,time
from bs4 import BeautifulSoup

Below code collects data from second page

In [3]:
a = []
view = []
rate = []
neg_count= 0
pos_count=0
neu_count=0
for i in range(120,600):
    response = requests.get('https://www.flipkart.com/apple-iphone-6-gold-32-gb/product-reviews/itmewxhuufbzchrn?pid=MOBEWXHUSBXVJ7NZ&lid=LSTMOBEWXHUSBXVJ7NZPXN7ZL&marketplace=FLIPKART&page='+str(i))
    soup = BeautifulSoup(response.text,'lxml')
    reviews = soup.find_all(class_='col _390CkK _1gY8H-')
    for i in reviews:
        a.append(i.get_text())
    for i in a:
        if int(i[0])<3:
            view.append(i[1:])
            rate.append("negative")
            neg_count+=1
        elif (int(i[0])==3 or int(i[0]) == 4):
            view.append(i[1:])
            rate.append("neutral")
            neu_count+=1
        else:
            view.append(i[1:])
            rate.append("positive")
            pos_count+=1


#Now this code collects data of first page

In [4]:
response = requests.get('https://www.flipkart.com/apple-iphone-6-gold-32-gb/product-reviews/itmewxhuufbzchrn?pid=MOBEWXHUSBXVJ7NZ&lid=LSTMOBEWXHUSBXVJ7NZPXN7ZL&marketplace=FLIPKART')
soup = BeautifulSoup(response.text,'lxml')
reviews = soup.find_all(class_='col _390CkK _1gY8H-')
for i in reviews:
    a.append(i.get_text())
for i in a:
    if int(i[0])<3:
        view.append(i[1:])
        rate.append("negative")
        neg_count+=1
    elif (int(i[0])==3 or int(i[0] == 4):
        view.append(i[1:])
        rate.append("neutral")
        neu_count+=1
    else:
        view.append(i[1:])
        rate.append("positive")
        pos_count+=1

SyntaxError: invalid syntax (<ipython-input-4-96cfa58ca53c>, line 11)

In [5]:
print("--------------------------------------------------------------")
print("neutral: ",neu_count)
print("----------------------------------------------------------------")
print("negative: ",neg_count)
print("----------------------------------------------------------------")
print("positive: ",pos_count)

--------------------------------------------------------------
neutral:  222580
----------------------------------------------------------------
negative:  183308
----------------------------------------------------------------
positive:  745162


## Creating the File

In [6]:
import csv

In [7]:
with open('rating1.csv','w',newline='',encoding='utf-8') as f:
    hello = csv.writer(f)
    hello.writerow(['reviews','ratings'])
    for i,j in zip(view,rate):
        hello.writerow([i,j])

In [8]:
import pandas as pd
csv = pd.read_csv('rating1.csv')
csv

,reviews,ratings
0,Hated it!I have not got a perfect phoneYou sho...,negative
1,Worst experience ever!WorstREAD MOREApurba De...,negative
2,Absolute rubbish!Very bad mobile don't buy I b...,negative
3,Best of appleStylish product.Was not sure of b...,positive
4,Simply awesomeGood oneREAD MORERanjith sagara...,positive
...,...,...
1151045,Perfect Machine but Battery Sucks!!!Classic Pr...,neutral
1151046,TerrificOrignal product. Order freely without ...,positive
1151047,Does the jobThanks for uREAD MOREarun kumarCe...,neutral
1151048,Best in the market!ExcellentREAD MOREAkramCert...,positive


In [9]:
import nltk
from nltk.corpus import stopwords
import string

In [10]:
def text_process(mess):
    # Remove punc
    # Remove Stop words
    # return clean list
    no_punc = [x for x in mess if x not in string.punctuation]
    no_punc = ''.join(no_punc)
    no_stop = [w for w in no_punc.split() if w.lower() not in stopwords.words('english')]
    return no_stop

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [12]:
from sklearn.model_selection import train_test_split
msg_train,msg_test,label_train,label_test = train_test_split(csv['reviews'],csv['ratings'],test_size=0.2,random_state=101)

In [13]:
from sklearn.pipeline import Pipeline

In [14]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
])

In [15]:
pipeline.fit(msg_train,label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7fc92bb5ae50>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [16]:
predictions = pipeline.predict(msg_test)

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(label_test,predictions))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     36804
     neutral       1.00      0.98      0.99     44589
    positive       0.99      1.00      1.00    148817

    accuracy                           0.99    230210
   macro avg       1.00      0.99      0.99    230210
weighted avg       0.99      0.99      0.99    230210



In [19]:
msg_test

933894    Best phoneBest phoneREAD MOREFlipkart Customer...
554197    Brilliantbest original product... delivered be...
631786    BrilliantGenuine n original.. luvng itREAD MOR...
392010    Slightly disappointedBattery Backup is Very Po...
350749    Terrible productBattery Sucks.Performance Suck...
                                ...                        
885080    Terrific purchaseGoodREAD MORESam KumarCertifi...
19659     Simply awesomeexcellentREAD MOREVivek JaiswalC...
470988    Terrible productThe product delivery was good,...
878603    Sexy rate beautiful phone apple n flipkart roc...
833923    Highly recommendedall apple products are worth...
Name: reviews, Length: 230210, dtype: object

In [23]:
pipeline.predict(['good bad worse'])

array(['negative'], dtype='<U8')